In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from scipy.stats import randint

from nltk.corpus import stopwords
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk import FreqDist, pos_tag
import re

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Integer, Real, Categorical

from scipy.stats import uniform, loguniform


In [63]:
data_raw = pd.read_csv('./data/allposts_struc.csv')
data_vecd = pd.read_csv('./data/merged_with_bigrams.csv')

In [65]:
#baseline accuracy
data_vecd.subreddit.value_counts(normalize=True)

relationship_advice    0.50775
pettyrevenge           0.49225
Name: subreddit, dtype: float64

# Modeling

In [66]:
#custom stopwords: 'english', + unique lems + 'im' just because
my_words = ['wa', 'ha', 'doe', 'a', 'im']
custom_stop = text.ENGLISH_STOP_WORDS.union(my_words)

In [67]:
pipe = Pipeline([
    ('cvec', CountVectorizer(stop_words=custom_stop) ),
    ('mnb', MultinomialNB() ),
])

pipe_params = {
    'cvec__max_features': randint(500,8000),
    'cvec__min_df': uniform(.01, .05),
    'cvec__max_df': [.90, .98],
    'cvec__ngram_range': [(1,1), (1,2), (1,3)],
    'mnb__alpha': uniform(.01,10)                    
}

In [68]:
X=data_raw['lem_selftext']
y=data_raw['subreddit']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.33)

In [69]:
vec_mnb_rs = RandomizedSearchCV(estimator = pipe, 
                                param_distributions=pipe_params, 
                                scoring = 'f1_weighted',
                                random_state=42,
                                n_iter=1000,
                                cv=5,
                                n_jobs=-1, verbose=1)

In [ ]:
vec_mnb_rs.fit(X_train, y_train)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/Users/helenmeigs/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   50.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  5.4min
